ollamaの起動

In [ ]:
# 1. 依存ツールのインストール
!apt-get update && apt-get install -y zstd pciutils

# 2. Ollamaのインストール
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
import time
import requests

# 共通設定（ここでモデル名を一括管理します）
# 使用したいモデル名（glm4, llama3, qwen2-vl など）をここに記述
MODEL_NAME = "qwen3-vl:4b"

In [ ]:
def setup_ollama_server(model_name):
    """サーバーを再起動し、指定されたモデルを準備する"""
    print("🔄 サーバーをリセット中...")
    !pkill ollama
    time.sleep(2)

    # サーバーをバックグラウンドで起動
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)

    print(f"⏳ モデル '{model_name}' を準備中...")
    subprocess.run(["ollama", "pull", model_name])

    # 接続確認
    try:
        res = requests.get("http://localhost:11434/api/tags")
        if res.status_code == 200:
            models = [m['name'] for m in res.json()['models']]
            print(f"✅ Ollama server is live! 利用可能モデル: {models}")
            return True
    except:
        print("❌ サーバーの起動に失敗しました。")
        return False

# 実行
setup_ollama_server(MODEL_NAME)

In [ ]:
import gradio as gr
import requests
import json
import base64

def encode_image_to_base64(image_path):
    """画像をBase64形式に変換する"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def chat_with_model(message, history):
    # message は {"text": "...", "files": ["path/to/file", ...]} という辞書形式で渡されます
    user_text = message["text"]
    user_files = message["files"]
    
    model = MODEL_NAME if 'MODEL_NAME' in globals() else "qwen3-vl:4b"
    url = "http://localhost:11434/api/chat"

    # メッセージの構築
    current_msg = {"role": "user", "content": user_text}
    
    # 画像ファイルがある場合、OllamaのAPI形式に合わせて追加
    if user_files:
        images_base64 = []
        for f in user_files:
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
                images_base64.append(encode_image_to_base64(f))
        
        if images_base64:
            current_msg["images"] = images_base64

    # 履歴の変換（簡易化のため、過去の画像は含めない構成）
    msgs = []
    for h in history:
        # history内の構造に合わせて調整が必要
        msgs.append({"role": "user", "content": h[0]})
        msgs.append({"role": "assistant", "content": h[1]})
    
    msgs.append(current_msg)

    payload = {
        "model": model,
        "messages": msgs,
        "stream": False
    }

    try:
        response = requests.post(url, json=payload, timeout=120)
        response.raise_for_status()
        return response.json()['message']['content']
    except Exception as e:
        return f"⚠️ エラー: {str(e)}"

# UIの起動
demo = gr.ChatInterface(
    fn=chat_with_model,
    title=f"🤖 {MODEL_NAME} Multimodal Chat",
    multimodal=True,  # これでファイル添付ボタンが表示されます
    textbox=gr.MultimodalTextbox(file_types=["image"]), # 画像のみに制限する場合
    theme="soft"
)

demo.launch(share=True, debug=True)

In [ ]:
# def launch_vscode_tunnel():
#     """VS Code Tunnelを起動し、GitHub認証URLを表示する"""
#     import os

#     # ファイルが存在しない場合のみダウンロード
#     if not os.path.exists("code"):
#         print("⏳ VS Code CLI をダウンロード中...")
#         !curl -Lk 'https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64' --output vscode_cli.tar.gz
#         !tar -xf vscode_cli.tar.gz

#     print("\n" + "="*50)
#     print("🚀 VS Code Tunnel を開始します...")
#     print("※ URLが表示されるまで 30秒〜1分ほどかかる場合があります。")
#     print("="*50 + "\n")

#     # 認証情報をリセットして確実に新しいリンクを出す
#     # 以前の `--provider` エラーを避けるため、最小限の引数で実行します
#     !./code tunnel user logout
#     !./code tunnel --accept-server-license-terms

In [ ]:
# !pkill code  # 残っているVS Codeプロセスを強制終了